### <font><font color="#2894FF">連線</font>    

In [1]:
import mysql.connector
from mysql.connector import Error
maxdb=mysql.connector.connect(
    host ='127.0.0.1',
    user = 'root',
    password='user'
)
cursor = maxdb.cursor()
print("success")

success


### <font><font color="#2894FF">執行SQL</font>

In [2]:
cursor.execute("SELECT Abstract FROM test.testcsv2;")


### <font><font color="#2894FF">把從資料庫讀入的Abstract放入到list中</font>

In [3]:
a=[]
for i in cursor:
    a.append(i)
    #print("\nPublicationDate: %s"%i)

In [4]:
a[0]

('A system includes a rocker arm assembly for operative engagement with a first and second cam. The assembly includes a first arm for operatively engaging the first cam for a first desired lift profile, a second arm for operatively engaging the second cam for a second desired lift profile, where the second arm includes a latch to engage the second arm with the first arm. The latch is responsive to supplied oil pressure and release oil pressure to switch between lift profiles. The system includes the latch coupled to the supplied or released oil pressure to engage the arms before the first and second arms are engaged with the base circle portion of each of the respective first and second cams.',)

In [6]:
import nltk
#nltk.download('punkt')
#nltk.download('stopwords')
import math
import string
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import *
from sklearn.feature_extraction.text import TfidfVectorizer


str1 = "".join(a[0])
str2 = "".join(a[1])
str3 = "".join(a[2])



### <font><font color="#2894FF">第一筆資料的Abstract</font>

In [7]:
str1

'A system includes a rocker arm assembly for operative engagement with a first and second cam. The assembly includes a first arm for operatively engaging the first cam for a first desired lift profile, a second arm for operatively engaging the second cam for a second desired lift profile, where the second arm includes a latch to engage the second arm with the first arm. The latch is responsive to supplied oil pressure and release oil pressure to switch between lift profiles. The system includes the latch coupled to the supplied or released oil pressure to engage the arms before the first and second arms are engaged with the base circle portion of each of the respective first and second cams.'

### <font><font color="#2894FF">第二筆資料的Abstract</font>

In [8]:
str2

'An oil supply control device includes: a memory which stores first master data constituted by predetermined control value; a hydraulic controller which outputs the control value to an adjusting device to cause a hydraulic pressure to coincide with a target hydraulic pressure; and a determination portion which determines whether or not a first difference between an output control value and the control value of the first master data lies within a predetermined allowable range, wherein the hydraulic controller starts to control the adjusting device with use of the control value of the first master data, when the first difference lies within the allowable range, and starts to control the adjusting device with use of the control value of second master data, when the first difference does not lie within the allowable range, the control value of the second master data causing the first difference to lie within the allowable range.'

### <font><font color="#2894FF">第三筆資料的Abstract</font>

In [9]:
str3

'A multiple variable valve lift apparatus may include a camshaft rotating by driving of an engine, a cam portion formed in a cylindrical shape having a hollow that the camshaft is inserted into, rotating together with the camshaft, configured to move along an axial direction of the camshaft, and forming a zero cam and a normal cam, a valve opening/closing device configured to be operated by at least one of the zero cam or the normal cam which are formed at the cam portion, an operating device disposed on an exterior circumference of the camshaft so as to move together with the cam portion, and a solenoid configured to selectively move the operating device along an axial direction of the camshaft, in which a journal, which has a radius being equal to a radius of the zero cam, is formed at the cam portion.'

### <font><font color="#2894FF">移除標點符號</font>

In [16]:
def get_tokens(text):
    lowers = text.lower()
    remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
    no_punctuation = lowers.translate(remove_punctuation_map)
    tokens = nltk.word_tokenize(no_punctuation)
    return tokens



### <font><font color="#2894FF">移除標點符號的結果</font>

In [11]:
tokens = get_tokens(str1)
count = Counter(tokens)
print (count.most_common(10))

[('the', 14), ('a', 8), ('second', 8), ('first', 7), ('arm', 6), ('for', 5), ('to', 5), ('includes', 4), ('and', 4), ('with', 3)]


### <font><font color="#2894FF">移除停詞(a the...)</font>

In [12]:
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed


### <font><font color="#2894FF">移除停詞結果</font>

In [13]:
tokens = get_tokens(str1)
filtered = [w for w in tokens if not w in stopwords.words('english')]
count = Counter(filtered)
print (count.most_common(10))

[('second', 8), ('first', 7), ('arm', 6), ('includes', 4), ('cam', 3), ('lift', 3), ('latch', 3), ('oil', 3), ('pressure', 3), ('system', 2)]


### <font><font color="#2894FF">把arm arms armed都算為arm</font>

In [14]:
tokens = get_tokens(str1)
filtered = [w for w in tokens if not w in stopwords.words('english')]
stemmer = PorterStemmer()
stemmed = stem_tokens(filtered, stemmer)

In [15]:
count = Counter(stemmed)
print(count.most_common(10))

[('arm', 8), ('second', 8), ('first', 7), ('engag', 6), ('includ', 4), ('cam', 4), ('oper', 3), ('lift', 3), ('profil', 3), ('latch', 3)]


### <font><font color="#2894FF">利用sklearn內建套件來計算tfidf</font>

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy 
import math

from sklearn.preprocessing import normalize

In [56]:
#str
#str1
#str2

In [18]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

### <font><font color="#2894FF">tfidf的結果</font>

In [19]:
vectorizer = TfidfVectorizer(sublinear_tf=False, stop_words='english', token_pattern="(?u)\\b\\w+\\b", smooth_idf=True, norm='l2')
tfidf = vectorizer.fit_transform([str1,str2,str3])
df_tfidf = pd.DataFrame(tfidf.toarray(),columns=vectorizer.get_feature_names(), index=['str', 'str1', 'str2'])

df_tfidf


,adjusting,allowable,apparatus,arm,arms,assembly,axial,base,cam,cams,camshaft,cause,causing,circle,circumference,closing,coincide,configured,constituted,control,controller,coupled,cylindrical,data,desired,determination,determines,device,difference,direction,disposed,does,driving,engage,engaged,engagement,engaging,engine,equal,exterior,formed,forming,having,hollow,hydraulic,include,includes,inserted,journal,latch,lie,lies,lift,master,memory,multiple,normal,oil,opening,operated,operating,operative,operatively,output,outputs,portion,predetermined,pressure,profile,profiles,radius,range,release,released,respective,responsive,rocker,rotating,second,selectively,shape,solenoid,starts,stores,supplied,supply,switch,target,use,value,valve,variable,zero
str,0.000000,0.00000,0.000000,0.476666,0.158889,0.158889,0.000000,0.079444,0.181259,0.079444,0.000000,0.000000,0.000000,0.079444,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.079444,0.000000,0.000000,0.158889,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.158889,0.079444,0.079444,0.158889,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.241678,0.000000,0.000000,0.238333,0.000000,0.000000,0.181259,0.000000,0.000000,0.000000,0.000000,0.181259,0.000000,0.000000,0.000000,0.079444,0.158889,0.000000,0.000000,0.046921,0.000000,0.181259,0.158889,0.079444,0.000000,0.00000,0.079444,0.079444,0.079444,0.079444,0.079444,0.000000,0.483356,0.000000,0.000000,0.000000,0.000000,0.000000,0.158889,0.000000,0.079444,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
str1,0.166567,0.22209,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055522,0.055522,0.000000,0.000000,0.000000,0.055522,0.000000,0.055522,0.555224,0.111045,0.000000,0.000000,0.277612,0.000000,0.055522,0.055522,0.168905,0.22209,0.000000,0.000000,0.055522,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.22209,0.000000,0.042226,0.000000,0.000000,0.000000,0.111045,0.111045,0.000000,0.277612,0.055522,0.000000,0.000000,0.042226,0.000000,0.000000,0.000000,0.000000,0.000000,0.055522,0.055522,0.032792,0.111045,0.084452,0.000000,0.000000,0.000000,0.22209,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.084452,0.000000,0.000000,0.000000,0.111045,0.055522,0.000000,0.055522,0.000000,0.055522,0.111045,0.388657,0.000000,0.000000,0.000000
str2,0.000000,0.00000,0.076424,0.000000,0.000000,0.000000,0.152848,0.000000,0.523100,0.000000,0.458543,0.000000,0.000000,0.000000,0.076424,0.076424,0.000000,0.229271,0.000000,0.000000,0.000000,0.000000,0.076424,0.000000,0.000000,0.000000,0.000000,0.174367,0.00000,0.152848,0.076424,0.000000,0.076424,0.000000,0.000000,0.000000,0.000000,0.076424,0.076424,0.076424,0.229271,0.076424,0.076424,0.076424,0.00000,0.076424,0.000000,0.076424,0.076424,0.000000,0.000000,0.000000,0.058122,0.000000,0.000000,0.076424,0.152848,0.000000,0.076424,0.076424,0.152848,0.000000,0.000000,0.000000,0.000000,0.180548,0.000000,0.000000,0.000000,0.000000,0.152848,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.152848,0.000000,0.076424,0.076424,0.076424,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.152848,0.076424,0.229271
